In [36]:
import pandas as pd
import numpy as np
import re
import psycopg2

In [2]:
df_2009 = pd.read_csv('d:/Gidromet/yangi_base.txt',header=None)

# Access and print the DataFrame

df_2009.columns=['wmo_id','year','month','day','hour','T_air','T_soil', 'T_dew_point', 'Partial_pressure', 'Relative_humidity', 
                        'saturation_deficit','Atmosphere_pressure_station','Atmosphere_pressure_sea_level','Trend_view',
                        'Trend_volume','Visibility','Cloudiness_all',  'Cloudiness_low','Ci','Ac','Cu','St','Fr','Height',
                        'Below','Weather_W','Weather_ww','Wind_right','Wind_speed','Precipitation','None']

# df_2009
df_2009['hour'] = pd.to_datetime(df_2009['hour'], format='%H:%M').dt.hour
df_2009['date'] = pd.to_datetime(df_2009[['year', 'month', 'day','hour']],errors='coerce')
dfj = df_2009['date'].dt.hour == 23
df_2009.loc[dfj, 'date'] += pd.Timedelta(days=1)
df_2=df_2009[['wmo_id','date','Wind_speed','Wind_right']]
df_2


C:\Users\User\AppData\Local\Temp\ipykernel_9392\1346918860.py:1: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,18,19,20,21,22,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2009 = pd.read_csv('d:/Gidromet/yangi_base.txt',header=None)


,wmo_id,date,Wind_speed,Wind_right
0,3736730,2000-01-01 23:00:00,1,100
1,3736730,2000-01-02 23:00:00,1,150
2,3736730,2000-01-03 23:00:00,0,0
3,3736730,2000-01-04 23:00:00,1,10
4,3736730,2000-01-05 23:00:00,0,0
...,...,...,...,...
4572443,4515830,2022-05-27 20:00:00,2,270
4572444,4515830,2022-05-28 20:00:00,2,90
4572445,4515830,2022-05-29 20:00:00,3,110
4572446,4515830,2022-05-30 20:00:00,2,90


In [3]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572448 entries, 0 to 4572447
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   wmo_id      int64         
 1   date        datetime64[ns]
 2   Wind_speed  object        
 3   Wind_right  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 139.5+ MB


In [4]:
df_2['Wind_right'] = pd.to_numeric(df_2['Wind_right'], errors='coerce')
df_2['Wind_right']=df_2['Wind_right'].astype(float)
df_2

C:\Users\User\AppData\Local\Temp\ipykernel_9392\4046320596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['Wind_right'] = pd.to_numeric(df_2['Wind_right'], errors='coerce')
C:\Users\User\AppData\Local\Temp\ipykernel_9392\4046320596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['Wind_right']=df_2['Wind_right'].astype(float)


,wmo_id,date,Wind_speed,Wind_right
0,3736730,2000-01-01 23:00:00,1,100.0
1,3736730,2000-01-02 23:00:00,1,150.0
2,3736730,2000-01-03 23:00:00,0,0.0
3,3736730,2000-01-04 23:00:00,1,10.0
4,3736730,2000-01-05 23:00:00,0,0.0
...,...,...,...,...
4572443,4515830,2022-05-27 20:00:00,2,270.0
4572444,4515830,2022-05-28 20:00:00,2,90.0
4572445,4515830,2022-05-29 20:00:00,3,110.0
4572446,4515830,2022-05-30 20:00:00,2,90.0


In [5]:
ranges = list(range(0, 370, 10))  # Ranges from 0 to 360 with a step of 10

# Add a new column to the dataframe indicating the range for each value
df_2['range'] = pd.cut(df_2['Wind_right'], bins=ranges, right=False, labels=ranges[:-1])
# df_2=df_2.drop('Wind_right')
# Display the dataframe
print(df_2.sample(20))

          wmo_id                date Wind_speed  Wind_right range
4418601  4395750 2013-05-03 02:00:00          0         0.0     0
3362340  4136940 2003-06-09 05:00:00          1        90.0    90
2475410  4086960 2016-11-15 17:00:00          0         0.0     0
3928660  4227090 2015-08-13 23:00:00          3        50.0    50
890031   3956390 2018-10-09 02:00:00          2       360.0   NaN
1973173  4066810 2019-05-24 05:00:00          2       270.0   270
953759   3966700 2020-02-05 20:00:00          1       270.0   270
917854   3966700 2006-10-10 08:00:00          1       202.0   200
942978   3966700 2016-06-27 11:00:00          2       192.0   190
2504146  4087160 2006-02-19 17:00:00        1.0        60.0    60
4059543  4255960 2019-10-20 11:00:00          2       180.0   180
3079309  4116990 2013-06-28 20:00:00          2       270.0   270
2350431  4086770 2016-01-12 11:00:00          3       110.0   110
347436   3836680 2018-03-12 20:00:00          5        90.0    90
962202   3

C:\Users\User\AppData\Local\Temp\ipykernel_9392\57134125.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['range'] = pd.cut(df_2['Wind_right'], bins=ranges, right=False, labels=ranges[:-1])


In [34]:
id_basa=pd.read_excel("d:/Gidromet/id_meteos.xlsx")
# id_basa_0=id_basa[['wmo_id','station_name']]
id_basa

,wmo_id,region,station_name,lat,lon,altitude,period_date
0,4087240,Andijon viloyati,Andijon,40.7166,72.2833,476.0,2000-2008
1,4077290,Andijon viloyati,Qo'rg'ontepa,40.6833,71.9333,433.0,2000-2008
2,4087160,Andijon viloyati,Yubeleyni,40.8024,71.5998,397.0,2000-2008
3,3976460,Buxoro viloyati,Buxoro,39.7666,64.4833,225.0,2000-2008
4,4096330,Buxoro viloyati,Jangeldi,40.8500,63.3333,208.0,2000-2008
...,...,...,...,...,...,...,...
72,4116900,Toshkent viloyati,Yangiyo'l,41.0167,69.0050,344.0,2000-2008
73,4156020,Xorazim viloyati,Gurlan,41.8490,60.3990,90.0,2000-2008
74,4146040,Xorazim viloyati,Xiva,41.4000,60.3830,97.0,2000-2008
75,4166060,Xorazim viloyati,Urganch,41.5830,60.6330,98.0,2000-2008


In [35]:
df2=df_2[df_2['wmo_id']==4395750]

january_df = df2[(df2['date'].dt.month == 1)]
# print(january_df)
average_wind_speed =january_df.groupby('range')['Wind_speed'].mean()
average_wind_speed=pd.DataFrame(average_wind_speed)
average_wind_speed=average_wind_speed.reset_index('range')

df1 = january_df.groupby(['range','wmo_id'])['range'].count().reset_index(name='frequency')

df1['total_frequency'] = df1.groupby('wmo_id')['frequency'].transform('sum')

df1['percentage'] = (df1['frequency'] / df1['total_frequency']) * 100
dff=pd.merge(average_wind_speed,df1,  on=['range'], how='inner')
# print(dff)


new_df = pd.DataFrame({
    'speed': dff['range'],
    'Combined': dff['Wind_speed'].round(2).astype(str)+ '/' + dff['percentage'].round(2).astype(str)   
})
# Print the new DataFrame
new_df1=new_df.T
# new_df1.insert(0,'Stansiya',str(id))
new_df1.to_excel("d:/Gidromet/natija.xlsx", header=True)
print(new_df1.head(30))

                 0         1          2          3          4         5   \
speed             0        10         20         30         40        50   
Combined  0.01/10.5  1.98/1.0  2.28/1.97  3.19/1.36  3.01/1.79  4.76/2.3   

                 6          7          8          9   ...         26  \
speed            60         70         80         90  ...        260   
Combined  5.48/3.37  6.76/3.91  7.13/4.62  5.3/10.92  ...  5.29/0.43   

                27         28         29         30         31         32  \
speed          270        280        290        300        310        320   
Combined  4.39/2.7  5.91/0.96  4.56/1.86  5.07/1.21  2.64/2.55  6.69/2.32   

                 33         34         35  
speed           330        340        350  
Combined  3.34/1.56  5.16/1.94  2.71/1.63  

[2 rows x 36 columns]


In [351]:
df=pd.read_csv("d:\Gidromet\shamol\wind_yer.txt",header=None,sep='\t',encoding='cp1251')
df.columns=['id','date','wind_speed','direction']
df['date']=pd.to_datetime(df['date'], format="%d.%m.%Y %H:%M")
df['id'] = df['id'].astype(str)
df

C:\Users\User\AppData\Local\Temp\ipykernel_9276\1631825536.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("d:\Gidromet\shamol\wind_yer.txt",header=None,sep='\t',encoding='cp1251')


,id,date,wind_speed,direction
0,38475,2022-12-20 20:00:00,0.0,"Штиль, безветрие"
1,38475,2022-12-20 17:00:00,1.0,"Ветер, дующий с востоко-северо-востока"
2,38475,2022-12-20 14:00:00,1.0,"Ветер, дующий с юго-запада"
3,38475,2022-12-20 11:00:00,0.0,"Штиль, безветрие"
4,38475,2022-12-20 08:00:00,1.0,"Ветер, дующий с северо-северо-запада"
...,...,...,...,...
3241261,UTNU,2019-01-19 16:30:00,10.0,"Ветер, дующий с юго-юго-запада"
3241262,UTNU,2019-01-19 16:00:00,10.0,"Ветер, дующий с юго-юго-запада"
3241263,UTNU,2019-01-19 15:30:00,7.0,"Ветер, дующий с юго-юго-запада"
3241264,UTNU,2019-01-19 15:00:00,8.0,"Ветер, дующий с юго-юго-запада"


In [276]:
df1=df[df['id']=='38475']
df1

,id,date,wind_speed,direction,Month,month
0,38475,2022-12-20 20:00:00,0.0,"Штиль, безветрие",12,12
1,38475,2022-12-20 17:00:00,1.0,"Ветер, дующий с востоко-северо-востока",12,12
2,38475,2022-12-20 14:00:00,1.0,"Ветер, дующий с юго-запада",12,12
3,38475,2022-12-20 11:00:00,0.0,"Штиль, безветрие",12,12
4,38475,2022-12-20 08:00:00,1.0,"Ветер, дующий с северо-северо-запада",12,12
...,...,...,...,...,...,...
40122,38475,2005-02-02 08:00:00,1.0,"Ветер, дующий с северо-востока",2,2
40123,38475,2005-02-02 02:00:00,0.0,"Штиль, безветрие",2,2
40124,38475,2005-02-01 20:00:00,1.0,"Ветер, дующий с севера",2,2
40125,38475,2005-02-01 14:00:00,1.0,"Ветер, дующий с запада",2,2


In [277]:
# df1 = df.groupby(pd.Grouper(key='date', freq='M')).mean()
# df1

In [262]:
wr = pd.read_csv('d:\Gidromet\shamol\wind_yer.txt', header=None, encoding='cp1251', sep='\t', dtype={0: str})
wr.columns = ['id', 'date', 'speed', '+']
wr['id'] = wr['id'].astype(str)
# wr = wr[wr['id'] == self.id]
wr['date'] = pd.to_datetime(wr['date'])

In [279]:
import pandas as pd

# Assuming your data is in a pandas DataFrame with columns 'date', 'wind_speed', 'directions'

# Step 1: Separate the data into individual months
df1['date'] = pd.to_datetime(df1['date'])  # Convert 'date' column to datetime type
df1['month'] = df1['date'].dt.month       # Extract the month component

# Step 2: Calculate the average for each month and direction
monthly_avg = df.groupby(['month', 'direction'])['wind_speed'].mean().reset_index()
print(monthly_avg)
# Step 3: Calculate the 18-year average for each direction
yearly_avg = monthly_avg.groupby('direction')['wind_speed'].mean()
print(yearly_avg)
# Print the 18-year average values for each direction
for direction, avg_value in yearly_avg.items():
    print(f"Direction: {direction}, 18-year average wind speed: {avg_value}")

C:\Users\User\AppData\Local\Temp\ipykernel_9276\2632055840.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['date'] = pd.to_datetime(df1['date'])  # Convert 'date' column to datetime type
C:\Users\User\AppData\Local\Temp\ipykernel_9276\2632055840.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['month'] = df1['date'].dt.month       # Extract the month component


     month                               direction  wind_speed
0        1                 Ветер, дующий с востока    3.144850
1        1  Ветер, дующий с востоко-северо-востока    2.735208
2        1     Ветер, дующий с востоко-юго-востока    2.941363
3        1                  Ветер, дующий с запада    2.856810
4        1    Ветер, дующий с западо-северо-запада    2.581649
..     ...                                     ...         ...
211     12              Ветер, дующий с юго-запада    3.056537
212     12         Ветер, дующий с юго-юго-востока    2.553997
213     12          Ветер, дующий с юго-юго-запада    2.617408
214     12                  Переменное направление    1.002554
215     12                        Штиль, безветрие    0.013414

[216 rows x 3 columns]
direction
Ветер, дующий с востока                   2.892840
Ветер, дующий с востоко-северо-востока    2.808345
Ветер, дующий с востоко-юго-востока       2.646681
Ветер, дующий с запада                    2.985764
Ветер,

In [304]:
df2=df[df['id']=='38475']
january_df = df2[(df2['date'].dt.month == 1)]
average_wind_speed = january_df.groupby('direction')['wind_speed'].mean()
df1 = january_df.groupby(['direction','id'])['direction'].count().reset_index(name='frequency')
df1['total_frequency'] = df1.groupby('id')['frequency'].transform('sum')
df1['percentage'] = (df1['frequency'] / df1['total_frequency']) * 100
print(df1)
print(average_wind_speed)

                                 direction     id  frequency  total_frequency  \
0                  Ветер, дующий с востока  38475        123             3164   
1   Ветер, дующий с востоко-северо-востока  38475        130             3164   
2      Ветер, дующий с востоко-юго-востока  38475         70             3164   
3                   Ветер, дующий с запада  38475        107             3164   
4     Ветер, дующий с западо-северо-запада  38475        119             3164   
5        Ветер, дующий с западо-юго-запада  38475        109             3164   
6                   Ветер, дующий с севера  38475        108             3164   
7           Ветер, дующий с северо-востока  38475        166             3164   
8            Ветер, дующий с северо-запада  38475        145             3164   
9    Ветер, дующий с северо-северо-востока  38475         71             3164   
10    Ветер, дующий с северо-северо-запада  38475        102             3164   
11                     Ветер

In [313]:
wr=pd.DataFrame(average_wind_speed)
wr=wr.reset_index()
wr

,direction,wind_speed
0,"Ветер, дующий с востока",1.577236
1,"Ветер, дующий с востоко-северо-востока",1.784615
2,"Ветер, дующий с востоко-юго-востока",1.457143
3,"Ветер, дующий с запада",1.579439
4,"Ветер, дующий с западо-северо-запада",1.588235
5,"Ветер, дующий с западо-юго-запада",1.761468
6,"Ветер, дующий с севера",1.564815
7,"Ветер, дующий с северо-востока",1.969880
8,"Ветер, дующий с северо-запада",1.606897
9,"Ветер, дующий с северо-северо-востока",1.943662


In [316]:
new_df = pd.DataFrame({
    'direction': wr['direction'],
    'percentage': df1['percentage'],
    'speed': wr['wind_speed']
    
})

# Print the new DataFrame
print(new_df)

                                 direction  percentage     speed
0                  Ветер, дующий с востока    3.887484  1.577236
1   Ветер, дующий с востоко-северо-востока    4.108723  1.784615
2      Ветер, дующий с востоко-юго-востока    2.212389  1.457143
3                   Ветер, дующий с запада    3.381795  1.579439
4     Ветер, дующий с западо-северо-запада    3.761062  1.588235
5        Ветер, дующий с западо-юго-запада    3.445006  1.761468
6                   Ветер, дующий с севера    3.413401  1.564815
7           Ветер, дующий с северо-востока    5.246523  1.969880
8            Ветер, дующий с северо-запада    4.582807  1.606897
9    Ветер, дующий с северо-северо-востока    2.243995  1.943662
10    Ветер, дующий с северо-северо-запада    3.223767  1.607843
11                     Ветер, дующий с юга    6.479140  1.892683
12             Ветер, дующий с юго-востока    2.939317  1.365591
13              Ветер, дующий с юго-запада    6.731985  2.183099
14         Ветер, дующий 

In [349]:
b=int(input("Stansiya id raqamini kiriting = "))
df2=df[df['id']==str(b)]
# print(df2)
df3=df2[df2['date'].dt.year<2021]
# print(df3[df3['date'].dt.year==2022])
january_df = df3[(df3['date'].dt.month == 1)]
average_wind_speed = january_df.groupby('direction')['wind_speed'].mean()
wr=pd.DataFrame(average_wind_speed)
wr=wr.reset_index()
df1 = january_df.groupby(['direction','id'])['direction'].count().reset_index(name='frequency')
df1['total_frequency'] = df1.groupby('id')['frequency'].transform('sum')

df1['percentage'] = (df1['frequency'] / df1['total_frequency']) * 100
# print(df1)
# print(average_wind_speed)
new_df = pd.DataFrame({
    'speed': wr['direction'],
    'Combined': df1['percentage'].round(1).astype(str) + '/' + wr['wind_speed'].round(1).astype(str)  
})
new_df1=new_df.T

# Print the new DataFrame
print(new_df1)
new_df1.to_excel("d:/Gidromet/natija.xlsx", header=True)
# datax=np.zeros((18, 16))
# jj=0

# for j in range(0,17):
#     print(j)
#     for i in range(len(wr4)):
#         print(i)
#         d= wr4.iloc[i]
#         yil=int(d[0].year-2005)
#         datax[yil,j]=d[1].round(2)
#         datax_yil=pd.DataFrame(datax)
# print(datax_yil)


Stansiya id raqamini kiriting = 38141
                               0                                       1   \
speed     Ветер, дующий с востока  Ветер, дующий с востоко-северо-востока   
Combined                 16.1/6.9                                 7.7/7.2   

                                           2                       3   \
speed     Ветер, дующий с востоко-юго-востока  Ветер, дующий с запада   
Combined                              6.7/6.2                 3.4/6.5   

                                            4   \
speed     Ветер, дующий с западо-северо-запада   
Combined                               3.0/6.3   

                                         5                       6   \
speed     Ветер, дующий с западо-юго-запада  Ветер, дующий с севера   
Combined                            2.3/6.0                 5.0/3.3   

                                      7                              8   \
speed     Ветер, дующий с северо-востока  Ветер, дующий с северо-запад

In [263]:
df1 =wr.groupby([pd.Grouper(key='date', freq='M'), 'id']).mean()
df1=df1.reset_index()
df1=df1.groupby([pd.Grouper(key='date', freq='Y'), 'id']).mean()
df1=df1.reset_index()
df1

,date,id,speed
0,2005-12-31,38023,1.567535
1,2005-12-31,38141,3.251018
2,2005-12-31,38146,4.423191
3,2005-12-31,38149,3.692389
4,2005-12-31,38178,2.785418
...,...,...,...
1114,2022-12-31,UTSK,2.862826
1115,2022-12-31,UTSS,3.710748
1116,2022-12-31,UTST,2.946767
1117,2022-12-31,UTTT,2.723154


In [264]:
# Группировка данных по сезонам и направлениям ветра
wr_seasonal = wr.groupby(['directions','id'])['directions'].count().reset_index(name='frequency')
# print(wr_seasonal)
dir_dict = {
    "Штиль, безветрие": None,
    "Переменное направление": None,
    "Ветер, дующий с севера": 0,
    "Ветер, дующий с северо-северо-востока": 22.5,
    "Ветер, дующий с северо-востока": 45,
    "Ветер, дующий с востоко-северо-востока": 67.5,
    "Ветер, дующий с востока": 90, 
    "Ветер, дующий с востоко-юго-востока": 112.5,
    "Ветер, дующий с юго-востока": 135, 
    "Ветер, дующий с юго-юго-востока": 157.5,
    "Ветер, дующий с юга": 180, 
    "Ветер, дующий с юго-юго-запада": 202.5,
    "Ветер, дующий с юго-запада": 225,
    "Ветер, дующий с западо-юго-запада": 247.5, 
    "Ветер, дующий с запада": 270, 
    "Ветер, дующий с западо-северо-запада": 292.5, 
    "Ветер, дующий с северо-запада": 315, 
    "Ветер, дующий с северо-северо-запада": 337.5,}

# Преобразование текстовых значений направлений ветра в числовые значения
wr_seasonal['dir'] = wr_seasonal['directions'].map(dir_dict)
wr_seasonal = wr_seasonal.dropna()


wr_seasonal['total_frequency'] = wr_seasonal.groupby('id')['frequency'].transform('sum')
wr_seasonal['percentage'] = (wr_seasonal['frequency'] / wr_seasonal['total_frequency']) * 100
# means = wr_seasonal['percentage'].values
print(wr_seasonal)

# Устанавливаем значения для направлений



                          directions     id  frequency    dir  \
0            Ветер, дующий с востока  38023       6736   90.0   
1            Ветер, дующий с востока  38141       7823   90.0   
2            Ветер, дующий с востока  38146       5054   90.0   
3            Ветер, дующий с востока  38149       5186   90.0   
4            Ветер, дующий с востока  38178       4127   90.0   
...                              ...    ...        ...    ...   
1244  Ветер, дующий с юго-юго-запада   UTSB       1335  202.5   
1245  Ветер, дующий с юго-юго-запада   UTSK       1946  202.5   
1246  Ветер, дующий с юго-юго-запада   UTSS        509  202.5   
1247  Ветер, дующий с юго-юго-запада   UTST       2565  202.5   
1248  Ветер, дующий с юго-юго-запада   UTTT       2728  202.5   

      total_frequency  percentage  
0               37170   18.122142  
1               44299   17.659541  
2               36370   13.896068  
3               43374   11.956472  
4               42998    9.598121  
...

In [255]:
wr2 = pd.merge(wr_seasonal, df1, on=['id'], how='inner')
wr2

,directions,id,frequency,dir,total_frequency,percentage,date,speed
0,"Ветер, дующий с востока",38023,6736,90.0,37170,18.122142,2005-12-31,1.567535
1,"Ветер, дующий с востока",38023,6736,90.0,37170,18.122142,2006-12-31,1.884988
2,"Ветер, дующий с востока",38023,6736,90.0,37170,18.122142,2007-12-31,1.944457
3,"Ветер, дующий с востока",38023,6736,90.0,37170,18.122142,2008-12-31,1.769756
4,"Ветер, дующий с востока",38023,6736,90.0,37170,18.122142,2009-12-31,2.259575
...,...,...,...,...,...,...,...,...
17824,"Ветер, дующий с юга",38415,1,180.0,1,100.000000,2018-12-31,NaN
17825,"Ветер, дующий с юга",38415,1,180.0,1,100.000000,2019-12-31,NaN
17826,"Ветер, дующий с юга",38415,1,180.0,1,100.000000,2020-12-31,NaN
17827,"Ветер, дующий с юга",38415,1,180.0,1,100.000000,2021-12-31,NaN


In [253]:
wr4=wr2[wr2['id']=='38457']
wr4

,directions,id,frequency,dir,total_frequency,percentage,date,speed
4288,"Ветер, дующий с востока",38457,5424,90.0,48162,11.261991,2005-12-31,1.169219
4289,"Ветер, дующий с востока",38457,5424,90.0,48162,11.261991,2006-12-31,1.094560
4290,"Ветер, дующий с востока",38457,5424,90.0,48162,11.261991,2007-12-31,1.236820
4291,"Ветер, дующий с востока",38457,5424,90.0,48162,11.261991,2008-12-31,1.424399
4292,"Ветер, дующий с востока",38457,5424,90.0,48162,11.261991,2009-12-31,1.370548
...,...,...,...,...,...,...,...,...
4571,"Ветер, дующий с юго-юго-запада",38457,1839,202.5,48162,3.818363,2018-12-31,1.281472
4572,"Ветер, дующий с юго-юго-запада",38457,1839,202.5,48162,3.818363,2019-12-31,1.256213
4573,"Ветер, дующий с юго-юго-запада",38457,1839,202.5,48162,3.818363,2020-12-31,1.280612
4574,"Ветер, дующий с юго-юго-запада",38457,1839,202.5,48162,3.818363,2021-12-31,1.426060


In [251]:
speed_values = wr4['speed'].values  


datax = np.reshape(speed_values[:288], (18, 16))


datax_yil = pd.DataFrame(datax)
datax_yil

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.169219,1.094560,1.236820,1.424399,1.370548,1.339051,1.223868,1.256038,1.267504,1.217537,1.329447,1.283596,1.242464,1.281472,1.256213,1.280612
1,1.426060,1.298372,1.169219,1.094560,1.236820,1.424399,1.370548,1.339051,1.223868,1.256038,1.267504,1.217537,1.329447,1.283596,1.242464,1.281472
2,1.256213,1.280612,1.426060,1.298372,1.169219,1.094560,1.236820,1.424399,1.370548,1.339051,1.223868,1.256038,1.267504,1.217537,1.329447,1.283596
3,1.242464,1.281472,1.256213,1.280612,1.426060,1.298372,1.169219,1.094560,1.236820,1.424399,1.370548,1.339051,1.223868,1.256038,1.267504,1.217537
4,1.329447,1.283596,1.242464,1.281472,1.256213,1.280612,1.426060,1.298372,1.169219,1.094560,1.236820,1.424399,1.370548,1.339051,1.223868,1.256038
5,1.267504,1.217537,1.329447,1.283596,1.242464,1.281472,1.256213,1.280612,1.426060,1.298372,1.169219,1.094560,1.236820,1.424399,1.370548,1.339051
6,1.223868,1.256038,1.267504,1.217537,1.329447,1.283596,1.242464,1.281472,1.256213,1.280612,1.426060,1.298372,1.169219,1.094560,1.236820,1.424399
7,1.370548,1.339051,1.223868,1.256038,1.267504,1.217537,1.329447,1.283596,1.242464,1.281472,1.256213,1.280612,1.426060,1.298372,1.169219,1.094560
8,1.236820,1.424399,1.370548,1.339051,1.223868,1.256038,1.267504,1.217537,1.329447,1.283596,1.242464,1.281472,1.256213,1.280612,1.426060,1.298372
9,1.169219,1.094560,1.236820,1.424399,1.370548,1.339051,1.223868,1.256038,1.267504,1.217537,1.329447,1.283596,1.242464,1.281472,1.256213,1.280612


In [224]:
b=int(input("Stansiya id raqamini kiriting = "))
wr3=wr2[wr2['id']==str(b)]
wr02=wr3[['date','speed','directions']]
wr4=wr02[wr02['date'].dt.month==1]
print(wr4)
datax=np.zeros((18, 16))
jj=0

for j in range(0,17):
    print(j)
    for i in range(len(wr4)):
        print(i)
        d= wr4.iloc[i]
        yil=int(d[0].year-2005)
        datax[yil,j]=d[1].round(2)
        datax_yil=pd.DataFrame(datax)
print(datax_yil)

# datax_yil.to_excel("D:/Gidromet/meteos_yog'in/" + str(id)+".xlsx", header=True)



Stansiya id raqamini kiriting = 38457
            date     speed                      directions
50608 2005-01-31  1.287356         Ветер, дующий с востока
50620 2006-01-31  1.113360         Ветер, дующий с востока
50632 2007-01-31  1.088710         Ветер, дующий с востока
50644 2008-01-31  1.229839         Ветер, дующий с востока
50656 2009-01-31  1.398340         Ветер, дующий с востока
...          ...       ...                             ...
54004 2018-01-31  1.169355  Ветер, дующий с юго-юго-запада
54016 2019-01-31  1.415323  Ветер, дующий с юго-юго-запада
54028 2020-01-31  1.202429  Ветер, дующий с юго-юго-запада
54040 2021-01-31  1.431452  Ветер, дующий с юго-юго-запада
54052 2022-01-31  1.457490  Ветер, дующий с юго-юго-запада

[288 rows x 3 columns]
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
7

IndexError: index 16 is out of bounds for axis 1 with size 16

In [258]:
import numpy as np

b = int(input("Stansiya id raqamini kiriting = "))
wr3 = wr2[wr2['id'] == str(b)]
wr02 = wr3[['date', 'speed', 'directions']]
# wr4 = wr02[wr02['date'].dt.month == 1]
print(wr02)

datax = np.zeros((18, 16))

for i in range(len(wr02)):
#     print(j)
    for j in range(16):
#         print(i)
        d = wr02.iloc[i]
        yil = int(d[0].year - 2005)
        datax[yil, j] = str(d[1].round(2))

datax_yil = pd.DataFrame(datax)
print(datax_yil.T)

Stansiya id raqamini kiriting = 38457
           date     speed                      directions
4288 2005-12-31  1.169219         Ветер, дующий с востока
4289 2006-12-31  1.094560         Ветер, дующий с востока
4290 2007-12-31  1.236820         Ветер, дующий с востока
4291 2008-12-31  1.424399         Ветер, дующий с востока
4292 2009-12-31  1.370548         Ветер, дующий с востока
...         ...       ...                             ...
4571 2018-12-31  1.281472  Ветер, дующий с юго-юго-запада
4572 2019-12-31  1.256213  Ветер, дующий с юго-юго-запада
4573 2020-12-31  1.280612  Ветер, дующий с юго-юго-запада
4574 2021-12-31  1.426060  Ветер, дующий с юго-юго-запада
4575 2022-12-31  1.298372  Ветер, дующий с юго-юго-запада

[288 rows x 3 columns]
      0     1     2     3     4     5     6     7     8     9     10    11  \
0   1.17  1.09  1.24  1.42  1.37  1.34  1.22  1.26  1.27  1.22  1.33  1.28   
1   1.17  1.09  1.24  1.42  1.37  1.34  1.22  1.26  1.27  1.22  1.33  1.28   
2   1.17